# Test `spatial_arrangment_1`

In [ ]:
using Pkg
Pkg.activate()
Pkg.instantiate()
#Carico le librerie utili all'implementazione ed alla fase di test
using Profile
using ProfileView
using BenchmarkTools #utile per il test
using SparseArrays
using LinearAlgebra
using Distributed
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation
using ViewerGL
GL = ViewerGL

  Activating project at `~/.julia/environments/v1.7`


ViewerGL

Preparazione input

In [ ]:
V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);


Testiamo il tempo d'esecuzione della funzione `spatial_arrangment`

In [ ]:
@btime spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)  # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione

Test tipicizzazione

In [ ]:
@code_warntype Lar.Arrangement.spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true) #con questa macro si valuta la tipicizzazione di una funzione

In [ ]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
  return rV, rEV, rFE
end

In [ ]:
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione

In [ ]:
@code_warntype spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true) #con questa macro si valuta la tipicizzazione di una funzione

# Ottimizazzione `frag_face`

La funzione `sptazial_arrangment_1` utilizza al suo interno, in un ciclo `for`, la funzione `frag_face` quindi una sua eventuale ottimizzazione è auspicabile. Tale funzione effettua la trasformazione in 2D delle facce fornite in input tramite il parametro sigma ed infine ogni faccia di questo parametro sigma si interseca con le facce `insp_index`. 

In [ ]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

In [ ]:
V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);


Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione.

In [ ]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

In [ ]:
@code_warntype frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

Ottimizzazione `frag_face`

In [ ]:
function frag_face_revisited(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
   @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    Threads.@threads for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

In [ ]:
@btime frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)

In [ ]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [ ]:
ProfileView.view()
Profile.print()

# Valutazione `spatial_arrangment_1`

La valutazione viene fatta aggiungendo al codice del precedente metodo `spatial_arrangment_1_rev` la funzione `frag_face_revisited`

In [ ]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face_revisited(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
  return rV, rEV, rFE
end

In [ ]:
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione

# Analisi e ottimizzazione `merge_vertices`

In [ ]:
function merge_vertices(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)
    # remove vertices congruent to a single representative
    todelete = []
    i = 1
    for vi in 1:vertsnum
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    for ei in 1:nedgenum
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    for e in 1:nedgenum
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    for fi in 1:nfacenum
        for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

In [ ]:
@btime merge_vertices($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

Ottimizazzione: si è scelto di usare la macro Julia `@async`. Tale macro avvolge un espressione in un `Task`e lo aggiunge alla coda dello scheduler della macchina locale.

In [ ]:
function merge_vertices_revisited(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #non è possibile utilizzare @async
        if !(vi in todelete)
            nearvs =  Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs =  setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #non è possibile utilizzare @async
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #non è possibile utilizzare @async
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

In [ ]:
@btime merge_vertices_revisited(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

Valutazione spatial_arrangment_1 con merge_vertices ottimizzato 

In [ ]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face_revisited(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices_revisited(rV, rEV, rFE)
  return rV, rEV, rFE
end

In [ ]:
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione

# Ottimizazzione `frag_face`

La funzione `sptazial_arrangment_1` utilizza al suo interno, in un ciclo `for`, la funzione `frag_face` quindi una sua eventuale ottimizzazione è auspicabile. Tale funzione effettua la trasformazione in 2D delle facce fornite in input tramite il parametro sigma ed infine ogni faccia di questo parametro sigma si interseca con le facce `insp_index`. 

In [ ]:
function frag_face(V, EV, FE, sp_idx, sigma)

    vs_num = size(V, 1)
    

	# 2D transformation of sigma face
    sigmavs = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV = V[sigmavs, :]
    sEV = EV[FE[sigma, :].nzind, sigmavs]
    M = Lar.Arrangement.submanifold_mapping(sV)
    tV = ([V ones(vs_num)]*M)[:, 1:3]  # folle convertire *tutti* i vertici
    sV = tV[sigmavs, :]
    # sigma face intersection with faces in sp_idx[sigma]
    for i in sp_idx[sigma]
        tmpV, tmpEV = Lar.Arrangement.face_int(tV, EV, FE[i, :])
       

        
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV, nEV, nFE = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    if nV == nothing ## not possible !! ... (each original face maps to its decomposition)
        return [], spzeros(Int8, 0,0), spzeros(Int8, 0,0)
    end
    nvsize = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3] ## ????
    
    return nV, nEV, nFE
end

In [ ]:
V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);


Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione.

In [ ]:
@btime frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

In [ ]:
@code_warntype frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

Ottimizzazione `frag_face`

In [ ]:
function frag_face_revisited(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64}, 
    sp_idx::Vector{Int64}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
   @views tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    Threads.@threads for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

In [ ]:
@btime frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)

In [ ]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face_revisited(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [ ]:
ProfileView.view()
Profile.print()

# Valutazione `spatial_arrangment_1`

La valutazione viene fatta aggiungendo al codice del precedente metodo `spatial_arrangment_1_rev` la funzione `frag_face_revisited`

In [ ]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face_revisited(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
  return rV, rEV, rFE
end

In [ ]:
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione

# Analisi e ottimizzazione `merge_vertices`

In [ ]:
function merge_vertices(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)
    # remove vertices congruent to a single representative
    todelete = []
    i = 1
    for vi in 1:vertsnum
        if !(vi in todelete)
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    for ei in 1:nedgenum
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    for e in 1:nedgenum
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    for fi in 1:nfacenum
        for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

In [ ]:
@btime merge_vertices($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

Ottimizazzione: si è scelto di usare la macro Julia `@async`. Tale macro avvolge un espressione in un `Task`e lo aggiunge alla coda dello scheduler della macchina locale.

In [ ]:
function merge_vertices_revisited(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #non è possibile utilizzare @async
        if !(vi in todelete)
            nearvs =  Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs =  setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #non è possibile utilizzare @async
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #non è possibile utilizzare @async
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

In [ ]:
@btime merge_vertices_revisited(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

Valutazione spatial_arrangment_1 con merge_vertices ottimizzato 

In [ ]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        @async for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face_revisited(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices_revisited(rV, rEV, rFE)
  return rV, rEV, rFE
end

In [ ]:
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) # Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale di una funzione